In [1]:
import pandas
from numpy import isnan

In [2]:
df = pandas.read_csv('csvs/raw/28.csv', index_col='frame')

In [3]:
df.sort_index(inplace=True)

In [4]:
(df['timer_value'] != -1).sum() / len(df)

0.7172966434835198

In [5]:
df = df[df['timer_value'] != -1]

In [6]:
def secant_is_ok_func(df):
    prev_value = df['timer_value'].shift(1)
    prev_index = df.index.to_series().shift(1)
    index_diff = df.index - prev_index
    seconds_slope = (df['timer_value'] - prev_value) / (index_diff / 29.97)
    secant_is_ok = seconds_slope.apply(lambda x: x > -1.5 or isnan(x))
    return secant_is_ok

In [7]:
while True: # filter by secant lines— should not fall > 1/sec / sec (ish)
    ok = secant_is_ok_func(df)
    df = df.loc[ok & ok.shift(-1), :]

    if (~ok).sum() <= 1 and (~secant_is_ok_func(df)).sum() <= 1:
        break

In [8]:
df.to_csv('csvs/cleaned/29|yBkNpkF_Q9A.csv')